In [31]:
# Import libraries and dependencies
import ccxt
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import talib as ta
import yfinance as yf 

from Utility_Functions import Functions

# Downloading the data from yfinance

### Set the currencies, date range and the interval for which you want the data

In [32]:
currs_list = [ 'BTC-AUD', 'ETH-AUD', 'XRP-AUD' , 'LTC-AUD', 'ADA-AUD', 'XLM-AUD', 'BCH-AUD']     #
start_date = '2019-06-01'
end_date = '2021-07-01'
interval = '1h'

### Call yfinance api

In [33]:
df_data = yf.download(currs_list, start= start_date, end= end_date, interval= interval, group_by= 'ticker')
df_data.head(2)


[*********************100%***********************]  7 of 7 completed


XLM-AUD                                          \
                               Open      High       Low     Close Adj Close   
2019-09-18 00:00:00+01:00  0.094861  0.095186  0.092567  0.092567  0.092567   
2019-09-18 01:00:00+01:00  0.092567  0.095486  0.092567  0.095366  0.095366   

                                       BTC-AUD                              \
                          Volume          Open          High           Low   
2019-09-18 00:00:00+01:00    0.0  14908.500000  14931.280273  14834.110352   
2019-09-18 01:00:00+01:00    0.0  14867.240234  14911.580078  14859.040039   

                                         ... XRP-AUD                    \
                                  Close  ...     Low   Close Adj Close   
2019-09-18 00:00:00+01:00  14858.330078  ...  0.4170  0.4200    0.4200   
2019-09-18 01:00:00+01:00  14901.099609  ...  0.4176  0.4419    0.4419   

                                      ADA-AUD                                \
                             Volume      Open      High       Low     Close   
2019-09-18 00:00:00+01:00       0.0  0.071778  0.072242  0.071261  0.071320   
2019-09-18 01:00:00+01:00  121342.0  0.071171  0.072918  0.071171  0.072717   

                                               
                          Adj Close    Volume  
2019-09-18 00:00:00+01:00  0.071320       0.0  
2019-09-18 01:00:00+01:00  0.072717  575734.0  

[2 rows x 42 columns]

### Split the data by currency and add it to the dictionary

Dictionary Structure: 
* Key= currency code 
* value = Dataframe conatining OHLCV data

In [34]:
df_data.shape

(15603, 42)

In [28]:
dict_ohlcv = {}

for curr in currs_list:

    df_ohlcv = df_data.loc[ :, ([curr]) ]
    curr_code = curr.replace('-', '/')

    df_ohlcv.columns = df_ohlcv.columns.droplevel(0)

    start_date = df_ohlcv.index[0].date().isoformat()
    end_date = df_ohlcv.index[-1].date().isoformat()
    num_records = (len(df_ohlcv))
    start_price = df_ohlcv.iloc[0]['Close']
    end_price = df_ohlcv.iloc[-1]['Close']

    print(f'Data summary for {curr_code}')
    print(f'    Start Date: {start_date}; End Date: {end_date}; NUmber of records: {num_records}')
    print(f'    Start Price: {start_price}; End Price: {end_price}')

    df_ohlcv.dropna(inplace=True)

    # Store the symbol name and history data in a dict 
    dict_ohlcv[curr_code] = df_ohlcv 

    print(f'Data for {curr_code} fetched and appended into the dictionary\n')


Data summary for BTC/AUD
    Start Date: 2020-05-31; End Date: 2021-06-30; NUmber of records: 9445
    Start Price: 14300.9189453125; End Price: 46277.2109375
Data for BTC/AUD fetched and appended into the dictionary

Data summary for ETH/AUD
    Start Date: 2020-05-31; End Date: 2021-06-30; NUmber of records: 9445
    Start Price: 352.8100891113281; End Price: 2853.845458984375
Data for ETH/AUD fetched and appended into the dictionary

Data summary for XRP/AUD
    Start Date: 2020-05-31; End Date: 2021-06-30; NUmber of records: 9445
    Start Price: 0.310027539730072; End Price: 0.8943524360656738
Data for XRP/AUD fetched and appended into the dictionary

Data summary for LTC/AUD
    Start Date: 2020-05-31; End Date: 2021-06-30; NUmber of records: 9445
    Start Price: 69.65292358398438; End Price: 186.65286254882812
Data for LTC/AUD fetched and appended into the dictionary

Data summary for ADA/AUD
    Start Date: 2020-05-31; End Date: 2021-06-30; NUmber of records: 9445
    Start Pr

# Compute the Technical Indicators using

In [29]:
fast_window = 5
slow_window = 15

df_all_data = pd.DataFrame() 
for curr, data in dict_ohlcv.items():

    df = Functions.add_tech_indicators(data, fast_window, slow_window)
    
    df['Currency'] = curr 
    df['Returns'] = df['Close'].pct_change()

    df.dropna(inplace=True)    
    df_all_data = df_all_data.append(df)

df_all_data.shape

(66009, 17)

In [30]:
df_all_data.to_csv('Resources/Training_data.csv', index=True)